# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [13]:
import pandas as pd
from pathlib import Path

RECIPES_PATH = Path("recipes.tsv")
TAGS_PATH    = Path("recipe-tags.tsv")

recipes = pd.read_csv(RECIPES_PATH, sep="\t")
tags    = pd.read_csv(TAGS_PATH,    sep="\t")

# Persona preferences
liked    = {"chocolate", "bacon", "spam", "cheese"}
disliked = {"kale", "quinoa", "vegan", "salad"}

# Build mapping: recipe_slug -> set of tags (avoid fillna with set())
tags_map = tags.groupby("recipe_slug")["recipe_tag"].apply(set).to_dict()

def score(slug: str) -> float:
    t = tags_map.get(slug, set())
    s = 0.5
    if liked & t:
        s += 0.5
    if disliked & t:
        s -= 0.5
    return float(min(1.0, max(0.0, s)))

ratings = recipes[["recipe_slug"]].copy()
ratings["rating"] = ratings["recipe_slug"].apply(score)

# Ensure at least 10 zeros and 10 ones (deterministic tie-breaking)
need_ones  = max(0, 10 - int((ratings["rating"] == 1.0).sum()))
need_zeros = max(0, 10 - int((ratings["rating"] == 0.0).sum()))
if need_zeros or need_ones:
    # stable ranking: by score then by original order
    tmp = ratings.assign(_ord=range(len(ratings))).sort_values(["rating","_ord"])
    if need_zeros:
        idx0 = tmp.index[:need_zeros]
        ratings.loc[idx0, "rating"] = 0.0
    if need_ones:
        idx1 = tmp.index[-need_ones:]
        ratings.loc[idx1, "rating"] = 1.0

ratings.to_csv("ratings.tsv", sep="\t", index=False)
print("Wrote ratings.tsv with shape:", ratings.shape)
print(ratings.head())


Wrote ratings.tsv with shape: (100, 2)
        recipe_slug  rating
0           falafel     0.0
1        spamburger     1.0
2  bacon-fried-rice     1.0
3   chicken-fingers     0.5
4       apple-crisp     0.5


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [14]:
import pandas as pd

recipes = pd.read_csv("recipes.tsv", sep="\t")
tags = pd.read_csv("recipe-tags.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

# one-hot encode recipe_tag per recipe_slug
tag_matrix = (
    pd.crosstab(tags["recipe_slug"], tags["recipe_tag"])
    .reindex(recipes["recipe_slug"], fill_value=0)
)

# add bias column = 1.0
features = tag_matrix.reset_index().rename_axis(None, axis=1)
features.insert(1, "bias", 1.0)  # after recipe_slug
features.rename(columns={"index":"recipe_slug"}, inplace=True)

# ensure same ordering as ratings.tsv (important)
features = features.set_index("recipe_slug").reindex(ratings["recipe_slug"]).reset_index()

# save
features.to_csv("features.tsv", sep="\t", index=False)
print("Wrote features.tsv with shape:", features.shape)
print(features.iloc[:3, :10])  # preview first few columns


Wrote features.tsv with shape: (100, 298)
        recipe_slug  bias  alfredo  almond  american  appetizer  appetizers  \
0           falafel   1.0        0       0         0          1           0   
1        spamburger   1.0        0       0         0          0           0   
2  bacon-fried-rice   1.0        0       0         0          0           0   

   apple  asiancuisine  asparagus  
0      0             0          0  
1      0             0          0  
2      0             0          0  


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [21]:
# YOUR CHANGES HERE
# Part 3 — Save `model.tsv` (recipe_tag, coefficient)

import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge

features = pd.read_csv("features.tsv", sep="\t")
ratings  = pd.read_csv("ratings.tsv",  sep="\t")

X_cols = features.drop(columns=["recipe_slug"]).columns

def ensure_series(theta_like) -> pd.Series:
    """Return coefficients as a pandas Series aligned to X_cols."""
    if isinstance(theta_like, pd.Series):
        # Reindex to X_cols just in case
        return theta_like.reindex(X_cols)
    # Handle numpy array / list / tuple
    arr = np.asarray(theta_like).ravel()
    if arr.shape[0] != len(X_cols):
        raise ValueError(f"Coefficient length {arr.shape[0]} != number of features {len(X_cols)}")
    return pd.Series(arr, index=X_cols, name="coefficient")

# Try to use theta_hat from the previous cell; otherwise recompute quickly
try:
    theta_series = ensure_series(theta_hat)  # may NameError if not defined
except NameError:
    X = features.drop(columns=["recipe_slug"]).values
    y = ratings.set_index("recipe_slug").loc[features["recipe_slug"], "rating"].values
    ridge = Ridge(alpha=1.0, fit_intercept=False, random_state=0)
    ridge.fit(X, y)
    theta_series = ensure_series(ridge.coef_)



Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [22]:
# Save model.tsv
model_df = theta_series.reset_index()
model_df.columns = ["recipe_tag", "coefficient"]
model_df.to_csv("model.tsv", sep="\t", index=False)

print("Wrote model.tsv with", len(model_df), "rows.")
print(model_df.head())

Wrote model.tsv with 297 rows.
  recipe_tag  coefficient
0       bias     0.433737
1    alfredo    -0.007756
2     almond    -0.015128
3   american     0.000796
4  appetizer     0.034916


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [23]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np

features = pd.read_csv("features.tsv", sep="\t")
model    = pd.read_csv("model.tsv", sep="\t")

# reconstruct theta in the same column order as features
theta = model.set_index("recipe_tag")["coefficient"]
X = features.drop(columns=["recipe_slug"])
theta = theta.reindex(X.columns).fillna(0.0)

est = X.values @ theta.values
est_df = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_estimate": est.astype(float)
})

est_df.to_csv("estimates.tsv", sep="\t", index=False)
print("Wrote estimates.tsv with shape:", est_df.shape)
print(est_df.head())


Wrote estimates.tsv with shape: (100, 2)
        recipe_slug  score_estimate
0           falafel        0.074464
1        spamburger        0.987882
2  bacon-fried-rice        0.983721
3   chicken-fingers        0.503772
4       apple-crisp        0.535028


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [24]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd

features = pd.read_csv("features.tsv", sep="\t")
ratings  = pd.read_csv("ratings.tsv", sep="\t")

X = features.drop(columns=["recipe_slug"]).values  # (n,d)
y = ratings.set_index("recipe_slug").loc[features["recipe_slug"], "rating"].values  # (n,)

n, d = X.shape
lam = 1.0
alpha = 2.0

A = X.T @ X + lam * np.eye(d)       # (d,d)
A_inv = np.linalg.inv(A)            # (d,d)
b = X.T @ y                         # (d,)
theta_hat = A_inv @ b               # (d,)

# UCB for each recipe
est = X @ theta_hat                 # (n,)
var = np.einsum("ij,jk,ik->i", X, A_inv, X)  # diag(X A_inv X^T)
ucb = est + alpha * np.sqrt(np.maximum(var, 0.0))

bounds_df = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_bound": ucb.astype(float)
})
bounds_df.to_csv("bounds.tsv", sep="\t", index=False)
print("Wrote bounds.tsv with shape:", bounds_df.shape)
print(bounds_df.head())


Wrote bounds.tsv with shape: (100, 2)
        recipe_slug  score_bound
0           falafel     1.840719
1        spamburger     2.882218
2  bacon-fried-rice     2.849878
3   chicken-fingers     2.297888
4       apple-crisp     2.311370


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [25]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd

features = pd.read_csv("features.tsv", sep="\t")
ratings  = pd.read_csv("ratings.tsv", sep="\t")

X_df = features.copy()
X = X_df.drop(columns=["recipe_slug"]).values  # (n,d)
slugs = X_df["recipe_slug"].tolist()
y_map = ratings.set_index("recipe_slug")["rating"].to_dict()

n, d = X.shape
lam = 1.0
alpha = 2.0

A = lam * np.eye(d)   # start with lambda*I
A_inv = np.linalg.inv(A)
b = np.zeros(d)

chosen = []
available = set(range(n))

for t in range(min(100, n)):
    theta = A_inv @ b  # current estimate
    # Compute UCB for all available arms
    X_av = X[list(available)]
    # variance term diag(X_av A_inv X_av^T)
    var = np.einsum("ij,jk,ik->i", X_av, A_inv, X_av)
    est = X_av @ theta
    p = est + alpha * np.sqrt(np.maximum(var, 0.0))
    # pick best
    idx_in_available = int(np.argmax(p))
    arm = list(available)[idx_in_available]
    x = X[arm]
    slug = slugs[arm]
    bound_val = float(p[idx_in_available])

    # observe reward from ratings
    r = float(y_map[slug])

    # record
    chosen.append((slug, bound_val, r))

    # update
    # Sherman–Morrison update for A_inv and b
    # A_new = A + x x^T
    x = x.reshape(-1,1)
    A_x = A_inv @ x
    denom = 1.0 + (x.T @ A_x)[0,0]
    A_inv = A_inv - (A_x @ A_x.T) / denom
    b = b + (r * x.flatten())
    available.remove(arm)

# save recommendations
recs = pd.DataFrame(chosen, columns=["recipe_slug","score_bound","reward"])
recs.to_csv("recommendations.tsv", sep="\t", index=False)
print("Wrote recommendations.tsv with shape:", recs.shape)
print(recs.head())


Wrote recommendations.tsv with shape: (100, 3)
     recipe_slug  score_bound  reward
0  apple-crumble     7.483315     0.5
1  ma-la-chicken     7.225922     0.5
2    quesadillas     7.215690     1.0
3          ramen     7.227970     0.5
4     spamburger     7.011715     1.0


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [26]:

ack_text = (
    "I used AI solely to clarify the intuition and formula for LinUCB upper confidence bounds "
    "(how the estimate and variance terms are combined). I implemented the code and completed "
    "the rest of the assignment myself."
)

with open("acknowledgments.txt", "w") as f:
    f.write(ack_text)

print("Wrote acknowledgments.txt")

Wrote acknowledgments.txt


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.